Proceso el dataset unificado de la universidad de Stuttgart y lo edito para mis fines.

Entre lo que hago esta:
1. Cambiar el formato de utf-8 a utf-16. ocupo el doble de espacio pero podre ver los emojis y otros caracteres especiales (13/06/2021: creo que esto al final nunca lo termine usando)
2. Procesar los escapes html que aparecen en los datasets crowdflower y emoint, como & quot ; y & amp ;

In [1]:
file = open("C:\\Users\\maram\\Proyecto-final\\modulos\\unify-emotion-datasets\\unified-dataset.jsonl","r",encoding="utf-8")

In [2]:
text = file.readlines()

In [3]:
len(text)

170687

In [4]:
import json
list_dics = list()

for line in text:
    list_dics.append(json.loads(line))

list_dics[0]

{'id': 0,
 'VAD': {'valence': None, 'arousal': None, 'dominance': None},
 'source': 'affectivetext',
 'text': 'Mortar assault leaves at least 18 dead',
 'emotions': {'joy': 0.0,
  'anger': 0.22,
  'sadness': 0.64,
  'disgust': 0.02,
  'fear': 0.6,
  'trust': None,
  'surprise': 0.0,
  'love': None,
  'noemo': None,
  'confusion': None,
  'anticipation': None,
  'shame': None,
  'guilt': None},
 'split': 'trial',
 'emotion_model': 'Ekman',
 'domain': 'headlines',
 'labeled': 'multi'}

In [5]:
list_dics[148031].get('text',None)

'I think @Sam_Canaday &amp; @KYLEJDOWSON must actually have to be working like me &amp; @dowson_brady because I havent got any snap chat videos today'

In [8]:
#Pruebo como hacer el escape de codigo html
from html import unescape

str = unescape(list_dics[148031].get('text',None));
str = str.replace('&', 'and')

print(str)

I think @Sam_Canaday and @KYLEJDOWSON must actually have to be working like me and @dowson_brady because I havent got any snap chat videos today


In [9]:
#Veo que modelos hay en el dataset
list_models = list()

for d in list_dics:
    list_models.append(d.get("source",None))

set_models = set(list_models)

set_models

{'affectivetext',
 'crowdflower',
 'dailydialog',
 'electoraltweets',
 'emoint',
 'emotion-cause',
 'fb-valence-arousal-anon',
 'grounded_emotions',
 'isear'}

In [10]:
#Hago una lista con los datos del dataset de mi eleccion
dataset = list()

for d in list_dics:
    if d.get("source") == 'electoraltweets':
        dataset.append(d)

len(dataset)

4056

In [50]:
#Defino una funcion que toma una lista de diccionarios y toma los
    #datos relevantes a un csv
#Escribir el csv de esta forma tiene el inconveniente de imprimir \n fila de por medio
import csv
from html import unescape

path = 'datasets/en/'

def write_csv(dataset):
    file_new = open(path+'datasetElectoral2.csv','w', encoding='utf-16')
    writer = csv.writer(file_new, delimiter='\t')
    writer.writerow(['text',
                   'joy',
                   'anger',
                   'sadness',
                   'disgust',
                   'fear',
                   'trust',
                   'surprise',
                   'anticipation'
                    ])
    for dic in dataset:
        writer.writerow([
                    unescape(dic['text']),#Agrego el encode porque algunos textos tenian comillas simples que Python no interpretaba bien
                    dic['emotions']['joy'],
                    dic['emotions']['anger'],
                    dic['emotions']['sadness'],
                    dic['emotions']['disgust'],
                    dic['emotions']['fear'],
                    dic['emotions']['trust'],
                    dic['emotions']['surprise'],
                    dic['emotions']['anticipation']
                    ]
                   )
    file_new.close()

write_csv(dataset)

## Adapto el dataset para utilizar Scikit-learn

In [51]:
import pandas as pd

Multinomial Naive Bayes me permite clasificar texto en varias categorías, aunque necesito sí o sí tener UNA SOLA variable Y.
Por eso adapto el csv del dataset.

Lo que hice fue alterar los datasets para que, en lugar de tener 1's y 0's según la aparición o no aparición de cada sentimiento, haya un solo campo 'emotion' que vaya del 0 al 8 indicando cual es el sentimiento que aparece. Siendo:

0. SIN EMOCION
1. JOY
2. ANGER
3. SADNESS
4. DISGUST
5. FEAR
6. TRUST
7. SURPRISE
8. ANTICIPATION

Esto no es una limitación en los datasets Electoral-Tweets y CrowdFlower, ya que estos solo tienen valores binarios para los sentimientos y solo indican la aparición de un solo sentimiento por texto.

In [52]:
list_emotions_P = ['No emo','Joy','Anger','Sadness','Disgust','Fear','Trust','Surprise','Anticipation']
list_emotions_E = ['No emo','Joy','Anger','Sadness','Disgust','Fear','Surprise']
dic_emotions_P = {'no emo':0, 'joy':1, 'anger':2, 'sadness':3, 'disgust':4, 'fear':5, 'trust':6, 'surprise':7, 'anticipation':8}
dic_emotions_E = {'No emo':0, 'Joy':1, 'Anger':2, 'Sadness':3, 'Disgust':4, 'Fear':5, 'Surprise':6}

In [123]:
import csv
path = 'datasets//en/'
df = pd.read_csv(path+'datasetElectoral2.csv', encoding='utf-16', sep='\t')

In [124]:
df.head(5)

,text,joy,anger,sadness,disgust,fear,trust,surprise,anticipation
0,"I'm a #Republican, but if I have to hear my mo...",0,0,0,1,0,0,0,0
1,Will Obama fire the person responsible for thi...,0,1,0,0,0,0,0,0
2,haha @DickMorrisTweet Romney is going to have ...,0,1,0,0,0,0,0,0
3,S/0 to my newest @freeboosieRS & vote for Obam...,0,0,0,0,0,0,0,1
4,Nicki Minaj Fucked Up With That Mitt Romney Li...,0,1,0,0,0,0,0,0


In [71]:
#Construyo el csv del nuevo dataset
file_new = open(path+'datasetElectoral2M.csv','w', encoding='utf-16')
writer = csv.writer(file_new, delimiter='\t')
writer.writerow(['text','emotion'])

#Arreglo para calcular la cantidad de apariciones de cada sentimiento en el dataset
emotions_array = [0,0,0,0,0,0,0,0,0]

for i in range(1,len(df)):
    row = df.loc[i]
    text = row["text"]
    #Busco la emocion caracteristica (se asume que solo se expresa una emocion)
    emotion = 0 #SIN EMOCION
    for j in range(1,9):
        if row[j] == 1:
            emotion = j
    emotions_array[emotion]+=1
    
    #Reemplazo caracteres invalidos por ? para adaptar el texto a utf-8
    #text = text.replace(u"\uFFFD", "?")
    writer.writerow([text,emotion])

file_new.close()
emotions_array

[1, 349, 569, 31, 1637, 91, 808, 251, 318]

El modelo de sentimientos de CrowdFlower es Ekman. Por esto los sentimientos TRUST y ANTICIPATION no tienen apariciones.

Algo interesante a señalar es la aparición de los sentimientos según el dataset. Los sentimientos más comunes en Electoral-Tweets son DISGUST y JOY, mientras que en CrowdFlower son NO_EMO, JOY y FEAR.

## Edito el dataset TEC

In [115]:
file = open(path+'datasetTEC.txt','r',encoding='utf-8')

text = file.readlines()

In [116]:
list_texts = list()

for t in text:
    new_text = t.replace(" \t:: ", "\t")
    new_text = new_text.replace(":\t", "\t")
    new_text = new_text.replace("\n", "")
    new_text = new_text.replace(",", "")
    list_texts.append(unescape(new_text))

print(list_texts[:3])

['145353048817012736\tThinks that @melbahughes had a great 50th birthday party :)\tsurprise', '144279638024257536\tComo una expresión tan simple una sola oración puede llegara dañarte tanto.\tsadness', '140499585285111809\tthe moment when you get another follower and you cheer.\tjoy']


In [117]:
file_new = open(path+'datasetTEC2.csv','w',encoding='utf-16')
writer = csv.writer(file_new, delimiter='\t')
writer.writerow(['ID','text','emotion'])
for line in list_texts:
    if line != '':
        l = line.split('\t')
        writer.writerow([l[0],l[1],l[2]])

file_new.close()

In [118]:
file_new = open(path+'datasetTEC2.csv','r',encoding='utf-16')

In [119]:
text = file_new.readlines()

In [167]:
import pandas as pd

df = pd.read_csv(path+'datasetUnificado.csv','r', encoding='utf-16', delimiter='\t')

type(df.emotion[34])

numpy.int64

In [95]:
file_new = open(path+'datasetTEC2M.csv','r',encoding='utf-16')
text = file_new.readlines()
text[2]

'Thinks that @melbahughes had a great 50th birthday party :),7\n'

# Construyendo el dataset unificado de los 4 subsets construidos anteriormente

In [18]:
lang = 'es'

In [22]:
#Uno los datasets mejorados (.*M.csv) en uno solo
import pandas as pd
path = 'datasets/united/'+lang+'/'
path_file1 = path+'datasetCrowdflower2-'+lang+'.csv'
path_file2 = path+'datasetElectoral2-'+lang+'.csv'
path_file3 = path+'datasetEmoInt2-'+lang+'.csv'
path_file4 = path+'datasetTec2-'+lang+'.csv'

df1 = pd.read_csv(path_file1, sep='\t', encoding='utf-16') #Ekman, un solo sentimiento por tweet
df2 = pd.read_csv(path_file2, sep='\t', encoding='utf-16') #Plutchick, un solo sentimiento por tweet
df3 = pd.read_csv(path_file3, sep='\t', encoding='utf-16') #Ekman, un solo sentimiento por tweet
df4 = pd.read_csv(path_file4, sep='\t', encoding='utf-16') #Ekman, un solo sentimiento por tweet

print('Abierto: '+path_file1)
print('Abierto: '+path_file2)
print('Abierto: '+path_file3)
print('Abierto: '+path_file4)

Abierto: datasets/united/es/datasetCrowdflower2-es.csv
Abierto: datasets/united/es/datasetElectoral2-es.csv
Abierto: datasets/united/es/datasetEmoInt2-es.csv
Abierto: datasets/united/es/datasetTec2-es.csv


In [23]:
import csv

name_file_new = path+'datasetUnificado-'+lang+'.csv'

#Construyo el csv del nuevo dataset unificado
writer_file_new = open(name_file_new,'w',encoding='utf-8')
writer = csv.writer(writer_file_new, delimiter='\t')

In [24]:
#Escribo el contenido del nuevo dataset unificado
writer.writerow(['text','emotion'])

i = 2
for i in range(len(df1)):
    if df1.text[i] != '\n':
        writer.writerow([df1.text[i],df1.emotion[i]])

i = 2
for i in range(len(df2)):
    if df2.text[i] != '\n':
        writer.writerow([df2.text[i],df2.emotion[i]])
        
i = 2
for i in range(len(df3)):
    if df3.text[i] != '\n':
        writer.writerow([df3.text[i],df3.emotion[i]])
        
i = 2
for i in range(len(df4)):
    if df4.text[i] != '\n':
        writer.writerow([df4.text[i],df4.emotion[i]])
        
file_new.close()
print('Creado: '+name_file_new)

Creado: datasets/united/es/datasetUnificado-es.csv
